In [1]:
%load_ext autoreload
%autoreload 2

## Calculate regional means of potential prognostic predictability

In [2]:
import xarray as xr
import numpy as np
import gfdl_utils as gu
import glob
from dask.diagnostics import ProgressBar
from information import *
from processing import *
from variance import *
from averaging import *
import os
import re

In [3]:
ppname = 'ice_month'
maskset = 'LME' # LME or global
metric = 'ppp'
comparison = 'e2c'
overwrite = True

In [4]:
# frequency = gu.core.get_timefrequency(get_pp(rootdir=ppeDict['rootdir_annual']),ppname)
frequency = gu.core.get_timefrequency(get_pp(),ppname)
startmonth = 1
filename = '.'.join([ppname,str(startmonth).zfill(2),'zarr'])
skill = xr.open_zarr(ppeDict['datasavedir']+'/processed/skill/'+metric+'.'+comparison+'/'+filename)
grid = xr.open_dataset(get_gridpath_from_ppname(ppname))

In [9]:
if maskset=='global':
    masks = generate_masks(grid)
elif maskset=='LME':
    masks = xr.open_dataset(ppeDict['pathLMEmask'])
    
### HACK
if ppname=='ice_month':
    masks = masks.rename({'xh':'xT','yh':'yT'})

In [10]:
savedir = ppeDict['datasavedir']+'/processed/skill_regionalmeans/'+metric+'.'+comparison+'/'
if not os.path.exists(savedir):
    os.makedirs(savedir)
for name,da in skill.items():
    print(name)
    tmp = calc_regionalmean_all(da,masks,weights=grid[get_areaname(grid)],verbose=True)
    savefilename = '.'.join([tmp.name,frequency,str(startmonth).zfill(2),maskset,'nc'])
    if os.path.isfile(savedir+savefilename):
        print(savefilename+' already exists.')
        if overwrite:
            print('Overwriting '+savefilename)
            with ProgressBar():
                tmp.to_netcdf(savedir+savefilename)
        else:
            continue
    else:
        print('Saving '+savefilename)
        with ProgressBar():
            tmp.to_netcdf(savedir+savefilename)

siconc
region: Baltic Sea
[########################################] | 100% Completed | 623.59 ms
region: Sea of Okhotsk
[########################################] | 100% Completed | 521.69 ms
region: North Sea
[########################################] | 100% Completed | 524.14 ms
region: Gulf of Alaska
[########################################] | 100% Completed | 523.49 ms
region: Labrador - Newfoundland
[########################################] | 100% Completed | 522.24 ms
region: Celtic-Biscay Shelf
[########################################] | 100% Completed | 523.87 ms
region: Sea of Japan
[########################################] | 100% Completed | 524.79 ms
region: Oyashio Current
[########################################] | 100% Completed | 523.98 ms
region: Scotian Shelf
[########################################] | 100% Completed | 521.71 ms
region: California Current
[########################################] | 100% Completed | 523.63 ms
region: Black Sea
[#################